In [7]:
import csv
import pandas

import os 
from pathlib import Path

cwd = Path(os.getcwd())
print("cwd: " + str(cwd))

data_file = cwd.parents[1] / 'data' / 'TraderLedger' / 'ledger.csv'
print(data_file)

cwd: /Users/vrajpandya/repo/OpenTrader/lab/Notebooks
/Users/vrajpandya/repo/OpenTrader/data/TraderLedger/ledger.csv


In [10]:
df = pandas.read_csv(data_file)

df.head()

,EntryID,OrderID,ClientID,Action,TotalQuantity,LimitPrice,TimeInForce,FilledQuantity,Symbol,Exchange,Currency,ExecutionID,ExecTime,AvgPrice,ExecutionSide,Commission,RealizedPNL,CommissionCurrency,Yield
0,0,3,0,BUY,0.003720,26888.0,Minutes,0.003720,BTC,SMART,USD,0,NaN,26870.0000,BOT,0.4,0,USD,0.0
1,1,4,0,BUY,0.003721,26878.0,Minutes,0.003721,BTC,SMART,USD,1,NaN,26875.8750,BOT,0.4,0,USD,0.0
2,2,5,0,BUY,0.003722,26868.0,Minutes,0.003722,BTC,SMART,USD,2,NaN,26862.9375,BOT,0.4,0,USD,0.0
3,3,6,0,BUY,0.003717,26908.0,Minutes,0.003717,BTC,SMART,USD,3,NaN,26898.6875,BOT,0.4,0,USD,0.0
4,4,7,0,BUY,0.003715,26918.0,Minutes,0.003715,BTC,SMART,USD,4,NaN,26917.1250,BOT,0.4,0,USD,0.0


In [11]:
import pymongo

# Establish a connection to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the trader_test_db database
db = client["trader_test_db"]

# Access the ConstantStepOffset_27888_10 collection
collection = db["ConstantStepOffset_27888_10"]

# Get all documents in the collection
docs = list(collection.find())

# Convert the list to a dictionary with indices as keys
docs_dict = {i: doc for i, doc in enumerate(docs)}

# Print an example document
print(docs_dict[0])

{'_id': ObjectId('65f63eba9851a426f91b5a2b'), 'baseline': 27888, 'step': 10, 'entry_context': {'order_id': 3, 'entry_id': 0}}


In [13]:
profit = 0

for index, row in df.iterrows():
    if row['Action'] == 'BUY':
        profit -= row['TotalQuantity'] * row['AvgPrice']
    elif row['Action'] == 'SELL':
        profit += row['TotalQuantity'] * row['AvgPrice']
    
    profit -= row['Commission'] * row['TotalQuantity'] * row['AvgPrice']

print(profit)


-17075.551299273604


In [16]:
config_dict = {
    "baseline" : 77000,
    "stepDelta": 10,
    "executionLimitOffset": 2000,
    "stateTransitionThreshold": 5,
    "orderQuantityInUSD": 100,
    "contract": {
        "symbol": "BTCUSD",
        "secType": "CRYPTO",
        "currency": "USD",
        "exchange": "SMART",
        "primaryExchange": "GEMINI"
    },
    "ledgerPath": "~/constant_step_offset_ledger_btc",
    "ledgerContextCollection": "ledgerContext"
}

baseline = config_dict['baseline']
stepDelta = config_dict['stepDelta']
orderQuantityInUSD = config_dict['orderQuantityInUSD']
executionLimitOffset = config_dict['executionLimitOffset']
numSteps = int(executionLimitOffset / stepDelta)


commission = 0.004
profit = 0

for step in range(1,numSteps + 1):
    btcQuantity = orderQuantityInUSD / (baseline - (step * stepDelta))
    profit += (baseline + (step * stepDelta)) * btcQuantity
    profit -= orderQuantityInUSD
    profit -= commission * orderQuantityInUSD

print(profit)

451.32159240103283


In [17]:
print(numSteps * orderQuantityInUSD)

print(100 * (profit) / (numSteps * orderQuantityInUSD))

20000
2.2566079620051642
